In [ ]:
%cd ..

In [ ]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.qimeraRBM import QimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD
from models.autoencoders.gumboltCaloCRBM import GumBoltCaloCRBM

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal
import dimod

In [ ]:
def init_model(cfg):
    model = GumBoltCaloCRBM(flat_input_size=[504],
                            train_ds_mean=0.,
                            activation_fct=torch.nn.ReLU(),
                            cfg=cfg)
    return model

In [ ]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="config-backup-oct-7-22")
    # this config file uses the chimera architecture

In [ ]:
model = init_model(cfg)
model.create_networks()

In [ ]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})
aux_crbm = QimeraRBM(n_visible=model.prior._n_visible, n_hidden=model.prior._n_hidden)
aux_crbm_sampler = PCD(batch_size=850, RBM=aux_crbm, n_gibbs_sampling_steps=5000)

In [ ]:
"""
Code to use GPU. Must specify which GPU unit to use here:
"""
GPU_NUM = 5
torch.cuda.set_device(5)
print("Using GPU {0}".format(torch.cuda.current_device()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Send sampler to GPU and define ZERO and MINUS_ONE in GPU
aux_crbm_sampler = aux_crbm_sampler.to(device)
ZERO = torch.tensor(0., dtype=torch.float).to(device)
MINUS_ONE = torch.tensor(-1., dtype=torch.float).to(device)

In [ ]:
def initialize_ising(n_vis, n_hid):
    """
    This function randomly initializes an Ising model with random J and h in the given ranges
    Inputs: nunmbers of visible and hidden nodes
    Output: Ising Weights and Biases
    * UPDATE: J is now drawn from a Gaussian distribution instead of a Uniform distribution
    """
    #wlim = [-0.5,0.5]
    #ising_weights = torch.nn.Parameter((wlim[1]-wlim[0])*torch.rand(n_vis, n_hid) + wlim[0], requires_grad=False) 
    ising_weights = torch.normal(0, 0.2, size=(n_hid, n_hid))
    hlim = [-2,2]
    ising_vbias = torch.nn.Parameter((hlim[1]-hlim[0])*torch.rand(n_vis)+hlim[0], requires_grad=False)
    ising_hbias = torch.nn.Parameter((hlim[1]-hlim[0])*torch.rand(n_hid)+hlim[0], requires_grad=False)
    return ising_weights, ising_vbias, ising_hbias

ising_weights, ising_vbias, ising_hbias = initialize_ising(1000,1000)

# Check to ensure J is in the valid DWAVE coupling range
min_J = np.min(ising_weights.cpu().detach().numpy())
max_J = np.max(ising_weights.cpu().detach().numpy())
print("J is between: {0} and {1}".format(min_J,max_J))

# the initialization below is necessary to run the function in
# the following block
dwave_energies = 0
dwave_samples = 0

In [ ]:
def beta_fixed_qpu(num_iterations:int=10, lr:float=0.01, beta_init=10., n_reads:int=100, qpu_sampler=None, 
                   aux_crbm_sampler=None, ising_weights=None, ising_vbias=None, ising_hbias=None, 
                   dwave_energies=0, dwave_samples=0, newDwave = 0):
    """Estimate the temperature associated with a given QPU configuration
    
    :param num_iterations (int) : Number of iterations
    :param lr (float) : Learning rate
    :param beta_init (float) : Initial estimate of the QPU temperature
    :param n_qpu_samples (int) : Number of QPU samples
    :param qpu_sampler
    
    """
    assert qpu_sampler is not None
    assert aux_crbm_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Auxiliary scaled RBM
    aux_crbm = aux_crbm_sampler.rbm
    aux_crbm_edgelist = aux_crbm.pruned_edge_list
    
    # Setup QPU control variables
    n_vis = len(aux_crbm.visible_qubit_idxs)
    n_hid = len(aux_crbm.visible_qubit_idxs)
    print("n_vis is {0} and n_hid is {1}\n".format(n_vis,n_hid))
    qubit_idxs = aux_crbm.visible_qubit_idxs+aux_crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(aux_crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(aux_crbm.hidden_qubit_idxs)}
    """
    Send Ising weights and biases to GPU is available
    """
    ising_weights = ising_weights.to(device)
    ising_vbias = ising_vbias.to(device)
    ising_hbias = ising_hbias.to(device)    
    """
    Also note that we are using a Chimera RBM architecture. Thus we have to transform 
    our Ising model by modifying the couplings to get the Chimera RBM architecture.
    We do this by masking the corresponding RBM weights of our ising_model
    and then reverting the rbm model back to the ising model. 
    """
    aux_crbm_weights = 4.*ising_weights # weights of the corresponding RBM
    aux_crbm_weights = aux_crbm_weights*aux_crbm.weights_mask # mask to get Chimera RBM architecture
    ising_weights = aux_crbm_weights*0.25 # revert back to ising model
    
    dwave_weights_np = -ising_weights.detach().cpu().numpy()
    print("J range = ({0}, {1})".format(np.min(dwave_weights_np),
                                        np.max(dwave_weights_np)))
    
    vbias_list = list(ising_vbias.detach().cpu().numpy())
    hbias_list = list(ising_hbias.detach().cpu().numpy())
    hVis = {v_qubit_idx:-vbias_list[visible_idx_map[v_qubit_idx]] for v_qubit_idx in aux_crbm.visible_qubit_idxs}
    hHid = {h_qubit_idx:-hbias_list[hidden_idx_map[h_qubit_idx]] for h_qubit_idx in aux_crbm.hidden_qubit_idxs}
    h = {**hVis,**hHid}
    J = {}
    for edge in aux_crbm_edgelist:
        if edge[0] in aux_crbm.visible_qubit_idxs:
            J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
        else:
            J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
    
    if (newDwave==1):
        response = qpu_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response, qubit_idxs) 
    dwave_samples = torch.tensor(dwave_samples, dtype=torch.float)  
    
    dwave_energy_exp = np.mean(dwave_energies, axis=0) # we use energies returned by DWAVE API to get expectation

    print("dwave_energy_exp : {0}".format(dwave_energy_exp))
    
    """
    Recall that in each iteration of the beta_eff* estimation we change the RBM parameters. However
    we still sample from the same RBM which has been initialized above. Hence, we keep track 
    of the original parameters by the following:
    """

    rbm_orig_vis = torch.nn.Parameter(2.*(ising_vbias - torch.sum(ising_weights, dim=1)), requires_grad=False).to(device)
    rbm_orig_hid = torch.nn.Parameter(2.*(ising_hbias - torch.sum(ising_weights, dim=0)), requires_grad=False).to(device)
    rbm_orig_weights = ising_weights*4
 
    classical_energies = [0]*num_iterations # classical energies is a 2D array containing 
                                            # classical energy distributions computed at each iteration
    
    for i in range(num_iterations):
        
        aux_crbm.weights = rbm_orig_weights*beta
        aux_crbm._visible_bias = torch.nn.Parameter(rbm_orig_vis*beta, requires_grad=False) 
        aux_crbm._hidden_bias = torch.nn.Parameter(rbm_orig_hid*beta, requires_grad=False)
        
        
        aux_crbm_sampler.rbm = aux_crbm
        
        aux_crbm_vis, aux_crbm_hid = aux_crbm_sampler.block_gibbs_sampling()
        
        """
        Finding energy using RBM method
        The code below computes the same ising energy energy expectation using RBM weights and biases.
        rbm_energy_exp is basically the same as aux_crbm_energy_exp and thus has been commented out
        """
        # ising_energy_rbm_params = ising_energy_rbm(rbm_orig_weights, rbm_orig_vis, rbm_orig_hid, aux_crbm_vis, aux_crbm_hid)
        # rbm_energy_exp = torch.mean(ising_energy_rbm_params, axis=0)        
        
        aux_crbm_vis = torch.where(aux_crbm_vis == ZERO, MINUS_ONE, aux_crbm_vis)
        aux_crbm_hid = torch.where(aux_crbm_hid == ZERO, MINUS_ONE, aux_crbm_hid)
        
        aux_crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, aux_crbm_vis, aux_crbm_hid)
        aux_crbm_energy_exps = -aux_crbm_energy_exp.detach().cpu().numpy()
        classical_energies[i] = aux_crbm_energy_exps
        aux_crbm_energy_exp = -torch.mean(aux_crbm_energy_exp, axis=0)
        
        print("aux_crbm_energy_exp : {0}, beta : {1} and epoch {2}".format(aux_crbm_energy_exp, beta, i+1))
        beta = beta - lr*(-float(aux_crbm_energy_exp)+float(dwave_energy_exp))
        betas.append(beta)
        
    return betas, dwave_energies, aux_crbm_energy_exps, ising_weights, J, h, qubit_idxs, classical_energies, dwave_samples

In [ ]:
# newDwave = 1 means we are getting new dwave samples and energies
# newDwave = 0 means we are using the previously computed dwave samples and energies
betas, dwave_energies,  aux_crbm_energy_exps, ising_weights, J, h, qubit_idxs, classical_energies, dwave_samples = beta_fixed_qpu(num_iterations=15,
                       lr=0.095,
                       beta_init=9.6,
                       n_reads=850,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler,
                       ising_weights = ising_weights,
                       ising_vbias = ising_vbias,
                       ising_hbias = ising_hbias,
                       dwave_energies = dwave_energies,
                       dwave_samples = dwave_samples,
                       newDwave = 1)

In [ ]:
plot_betas(betas)

In [ ]:
def plot_energies(energies1, energies2, binwidth):
    """
    Plot the energies of the samples produced by the histograms   
    UPDATE: bin now found using bin boundaries
    """
    fig, ax = plt.subplots(figsize=(40, 16))
    data = np.concatenate((energies1,energies2), axis=0)
    bins =  np.arange(min(data), max(data) + binwidth, binwidth)
    ax.hist(energies1, bins=bins, label = "dwave energies")  
    ax.hist(energies2, bins=bins, label = "auxiliary RBM energies", alpha=0.5) 
    plt.legend(loc='upper right', fontsize=30)
    ax.set_xlabel("Energy", fontsize=60)
    ax.set_ylabel("Frequency", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.grid(True)
    
    plt.show()
    plt.close()

In [ ]:
for i in range(len(classical_energies)):
    plot_energies(dwave_energies, classical_energies[i], 0.05)

In [ ]:
import time
# this will give dwave energy distributions in 58 second intervals 
def getDwavePlots(num_iter, n_reads, ising_weights, ising_vbias, ising_hbias):
    dwave_array = [0]*num_iter
    for i in range(num_iter):
        betas, dwave_energies,  aux_crbm_energy_exps, ising_weights, J, h, qubit_idxs, classical_energies, dwave_samples = beta_fixed_qpu(num_iterations=1,
                               lr=0.05,
                               beta_init=9.27, 
                               n_reads=n_reads,
                               qpu_sampler=qpu_sampler,
                               aux_crbm_sampler=aux_crbm_sampler,
                               ising_weights = ising_weights,
                               ising_vbias = ising_vbias,
                               ising_hbias = ising_hbias,
                               dwave_energies = 0,
                               dwave_samples = 0,
                               newDwave = 1) 
        dwave_array[i] = dwave_energies
        if (i!=num_iter-1):
            time.sleep(58)
    return dwave_array
dwave_array = getDwavePlots(4,2000, ising_weights, ising_vbias, ising_hbias)